# Golden Olympic 2012 Post Boxes

Following the Tokyo 2020 Olympics I thought it would be interesting to map all the post boxes that were painted golden after the London 2012 using data from [OpenStreetMap](https://www.openstreetmap.org/#map=19/49.43182/-2.35568). I've selected the post boxes by finding everything coloured gold in the British Isles, then filtering by post boxes, as there are fewer gold items than post boxes in OpenStreetMap.

There is clearly some data missing, both missing post boxes and information. I think it would be nice to see this filled in!

In [1]:
import requests
import json
from types import SimpleNamespace


import openstreetmap_mapping as osm

import pandas as pd

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook

from bokeh.models import OpenURL, TapTool, Title

output_notebook()

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
class Tags(object):
    
    def __init__(self, *initial_data, **kwargs):
              
        for dictionary in initial_data:
            
            for key in dictionary:

                if isinstance(dictionary[key], dict):
                    if len(dictionary[key])>1:
                        setattr(self, 'value', key)
                        setattr(self, key, Tags(dictionary[key]))
                    else:
                        setattr(self, key, Tags(dictionary[key]))
                else:
                    setattr(self, key, dictionary[key])
            
    def __repr__(self):
        return self.value.replace("__",":")
    
    def __str__(self):
        return self.value.replace("__",":")

In [4]:
kvs = osm.toolkit.get_osm_kvs()

In [5]:
tags = Tags(kvs)

In [6]:
area_Guernsey = "(49.4096, -2.6875,49.5194, -2.4901)"
area_Ealing = "(51.487, -0.351, 51.5335, -0.2655)"
area_britishisles = "(48.327, -12.063, 61.428, 1.78)"
area_Glasgow = "(55.5683, -4.9507, 56.1318, -3.6639)"
area_Whitelee = "(55.5, -4.4364, 55.7591, -4.1)"
area_Wales = "(51.221, -5.559, 53.619, -2.845)"
area_Shetlands = "(59.8034, -2.4527, 60.9331, -0.4257)"
area_London = "(51.448, -0.2554, 51.6061, 0.0714)"

In [7]:
df_gold = osm.toolkit.get_osm_data(key=tags.colour,tag='gold',area=area_britishisles)


In [8]:
df_postboxes = df_gold[df_gold['amenity']=='post_box']

In [9]:
df_postboxes.columns=df_postboxes.columns.str.replace(":", "__")

In [10]:
df_postboxes.columns

Index(['type', 'id', 'lat', 'lon', 'tags', 'center', 'members', 'amenity',
       'colour', 'post_box__type', 'ref', 'collection_times', 'operator',
       'post_box__apertures', 'royal_cypher', 'description', 'drive_through',
       'inscription', 'postal_code', 'note', 'opening_hours', 'brand',
       'brand__wikidata', 'brand__wikipedia', 'operator__wikidata',
       'operator__wikipedia', 'post_box__design', 'source', 'olympic_gold',
       'olympic_gold__note', 'collection_plate', 'ele', 'olympics__2012',
       'note_1', 'old_ref', 'mapillary', 'collection_times_checked', 'name',
       'commemorates', 'tactile_writing__braille__de', 'note__colour',
       'website', 'ref__duplicates', 'olympics_gold', 'post_box__mounting',
       'disused__amenity', 'booth', 'covered', 'man_made', 'material',
       'cycle_network', 'network', 'route', 'key', 'tag'],
      dtype='object')

In [11]:
df_postboxes = df_postboxes.fillna("")

In [12]:
df_postboxes['Olympics2012'] = (df_postboxes['description'].astype(str)
                                +df_postboxes['inscription'].astype(str)
                                +df_postboxes['note'].astype(str)
                                +df_postboxes['olympic_gold'].astype(str)
                                +df_postboxes['olympic_gold__note'].astype(str)
                                +df_postboxes['olympics__2012'].astype(str)
                                +df_postboxes['commemorates'].astype(str)
                                +df_postboxes['olympics_gold'].astype(str)
                                )

In [13]:
df_postboxes['osm_link'] = r'https://www.openstreetmap.org/node/'+df_postboxes['id'].astype(str)

In [16]:
output_file("GoldenPostbox.html")

kwargs_for_figure = {"tooltips":[("OSM link", "@osm_link"),
                                 ("Note", "@Olympics2012"),
                                 ("Collection", "@collection_times"),
                                 ("Type","@post_box__type"),
                                 ("Design","@post_box__design"),
                                 ("Cypher","@royal_cypher"),
                                 ("(lat,lon)", "@coordinates")
                                ],
                     "sizing_mode":"scale_both",
                    }

p = osm.toolkit.plot_latlon(df_postboxes,
                            tile_name="OpenMap",
                            kwargs_for_figure=kwargs_for_figure,
                            kwargs_for_marker={"marker":"square_dot","fill_color":"gold",'legend_label':'golden postbox'})

p.legend.visible  = False

url = "@osm_link"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

p.title.text = 'Golden 2012 Olympic Postboxes'
p.title.text_color = "gold"
p.title.background_fill_color = "red"

p.add_layout(Title(text="Created with OpenStreetMap data", align="left"), "left")

show(p)